In [1]:
# Se importan las librerías que se van a usar
import requests
import time
import re
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import Edge
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.by import By


# Esta es la web donde aparecen las películas ganadoras y nominadas a mejor película en
# los óscar durante la década de 2010's. La usaremos como punto de partida para
# extraer toda la información
url = "https://www.filmaffinity.com/es/awards3.php?award_id=academy_awards&decade=2010"

# Modificamos el User-Agent para evitar posibles problemas de bloqueo. Se comprueba
# el archivo robots.txt y a priori no deberían bloquearnos
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,\
    */*;q=0.8",
    "Accept-Encoding": "gzip, deflate, sdch, br",
    "Accept-Language": "en-US,en;q=0.8",
    "Cache-Control": "no-cache",
    "dnt": "1",
    "Pragma": "no-cache",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36\
     (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36"
}

# URL en la que se hace el login en la web
url_login = "https://www.filmaffinity.com/es/account.ajax.php?action=login"

# Se indican los parámetros que son necesarios para hacer el login
login_data =  {
    'postback':1,
    'rp':url,
    'username':'Gallardola',
    'password':'P4ssw0rd!'
}

In [2]:
def get_page(url):
    # Esta función se implementa para extraer el contenido de una URL. Utiliza
    # las cookies de sesión y guarda un tiempo prudencial entre peticiones que
    # es proporcional al tiempo de respuesta del servidor
    try:
        soup = None
        t0 = time.time()
        page = session.get(url, headers=headers)
        delay = time.time() - t0
        if page.status_code == 200:
            soup = BeautifulSoup(page.content)
        else:
            print("Ha habido algún problema al descargar el contenido.\n")
        time.sleep(20*delay)
    except requests.exceptions.Timeout:
        pass
    except requests.exceptions.RequestException:
        pass
    return soup


def find_movie_info(movie_info):
    # Extrae la información que nos interesa de la película y la almacena en
    # una lista
    wanted_data = ["Título original", "Año", "Duración", "País", "Dirección",
                   "Reparto", "Género", "Sinopsis"]
    dt_list = movie_info.find_all("dt")
    dd_list = movie_info.find_all("dd")

    info = [dd_list[i] for i in range(len(dt_list)) if dt_list[i].
            get_text(strip=True) in wanted_data]
    return info


def get_rating_info():
    # Si la página tiene votaciones y una nota la extrae, si no devuelve vacío
    rating_info = []
    result = []
    try:
        rating_info = soup.find(id="rat-avg-container").find_all('div')
        result.append(rating_info[0].get_text(strip=True))
        result.append(rating_info[1].span.get_text(strip=True))
    except AttributeError:
        result = ["", ""]
    return result


def load_requests(source_url):
    # descarga el cartel o poster de la película, guarda un tiempo prudencial
    # entre descargas proporcional a la respuesta del servidor
    t0 = time.time()
    r = session.get(source_url, headers=headers, stream=True)
    delay = time.time() - t0
    if r.status_code == 200:
        aSplit = source_url.split('/')
        ruta = "C:/Users/a091b/images_FA/"+aSplit[len(aSplit)-1]
        print(ruta)
        output = open(ruta, "wb")
        for chunk in r:
            output.write(chunk)
        output.close()
    time.sleep(10*delay)

In [3]:
# Se utilizan las cookies de sesión para hacer el login
session = requests.Session()
r = session.post(url_login, data=login_data, headers=headers)
# Se comprueba si el servidor da algún error
for i in r.history:
    if r.status_code == 401:
        print("Error!")

In [4]:
# a través de la URL de inicio, se consiguen las URLs para todas las decadas
# en las que se han celebrado los premios Oscars: 1920s a 2020s
decade_soap = get_page(url)
URL_decades = []
decades = decade_soap.find("table", class_='decades').find_all("a")
for i in decades:
    URL_decades.append(i.get("href"))
# Se elimina la última URL que nos conduce a todos los premios
URL_decades.pop()

'https://www.filmaffinity.com/es/all_awards.php'

In [5]:
# Se crea una lista donde se almacenarán las URL de todas las películas
URL_films = []

# Se busca en cada década las URLs de las películas premiadas y nominadas
for dec in URL_decades:
    main_soup = get_page(dec)
    # Dentro del div con el atributo class especificado se encuentran las
    # ganadoras del premio
    awarded_films = main_soup.body.find_all(
        "div", class_="aw-mc2 big-poster winner-border")

    # Dentro del div con este atributo class se encuentran las películas
    # que han sido nominadas pero no ganaron el premio
    nominated_films = main_soup.body.find_all(
        "div", class_="aw-mc2 big-poster")

    # Observando su estructura, la URL a la web con toda la información
    # de la película se encuentra dentro de la etiqueta a, en el atributo
    # href, por lo que iteramos para almacenarlas en URL_films
    for i in range(len(awarded_films)):
        URL_films.append(awarded_films[i].a.get("href"))
    for i in range(len(nominated_films)):
        URL_films.append(nominated_films[i].a.get("href"))

In [7]:
# Se crea un índice que se usara como key del diccionario donde se almacenará
# la información
index = 0
# Se crea el diccionario
data_dict = {}
# Se crea el driver para Selenium con el navegador MS Edge
driver = webdriver.Edge()
for i in URL_films:
    # Se recorren todas las URLs con las películas buscando la información
    # que se quiere almacenar en el dataset
    soup = get_page(i)
    # Desplegamos los premios ocultos en caso de que los haya utilizando
    # Selenium
    driver.get(i)
    try:
        show_hidden_awards = driver.find_element_by_id('show-all-awards')
        time.sleep(5)
        show_hidden_awards.click()
    except NoSuchElementException:
        pass
    except ElementClickInterceptedException:
        # Ejecutamos Javascript (más eficiente) si el delay no es suficiente
        driver.execute_script("arguments[0].click();", show_hidden_awards)

    # Separamos la web en las 2 zonas donde existe información de interés
    movie_info = find_movie_info(soup.find(class_="movie-info"))
    rating_info = get_rating_info()

    # Aquellos atributos que son una lista de valores los generamos previamente
    actor_list = [actor.get_text(strip=True) for actor in movie_info[5].
                  div.find_all("span")[1::2]]
    genre_list = [genre.get_text(strip=True) for genre in movie_info[6].
                  find_all("a")]

    # Almacenamos el voto personal de la película, si el valor es "-1" quiere
    # decir que no se ha visto, por lo que se deja vacío
    my_vote = soup.find("div", class_='rate-movie-box').attrs[
        'data-user-rating']

    # Construimos el diccionario con todos los datos para una película
    data_dict[index] = {
        "ID_FA": re.findall('\\d+', soup.find(
            "meta", property="og:url").get("content"))[0],
        "title": soup.find("h1", id="main-title").span.get_text(strip=True),
        "original_title": movie_info[0].get_text(strip=True),
        "year": movie_info[1].get_text(strip=True),
        "duration": movie_info[2].get_text(strip=True),
        "country": movie_info[3].get_text(strip=True),
        "director": movie_info[4].div.a.get_text(strip=True),
        "actors": actor_list,
        "genre": genre_list,
        "description": movie_info[7].get_text(strip=True),
        "awards": driver.find_element_by_css_selector(
            '.margin-top.movie-info dd').text.split('\n'),
        "average_rating": rating_info[0],
        "rating_votes": rating_info[1],
        "poster": soup.find(id="movie-main-image-container").a.get("href"),
        "my_vote": my_vote if my_vote != "-1" else ""
    }
    index += 1

In [8]:
# Se crea un dataframe a partir del diccionario
df = pd.DataFrame().from_dict(data_dict, orient='index')
df.head().append(df.tail())

,ID_FA,title,original_title,year,duration,country,director,actors,genre,description,awards,average_rating,rating_votes,poster,my_vote
0,602893,La melodía de Broadway,The Broadway Melody,1929,110 min.,Estados Unidos,Harry Beaumont,"[Charles King, Anita Page, Bessie Love, Jed Pr...","[Musical, Romance, Comedia]",Queenie y Hank son dos hermanas que buscan tri...,[1928: Oscar: Mejor película. 3 nominaciones],"5,9",482,https://pics.filmaffinity.com/the_broadway_mel...,
1,415986,Alas,Wings,1927,138 min.,Estados Unidos,William A. Wellman,"[Clara Bow, Charles 'Buddy' Rogers, Richard Ar...","[Bélico, Acción, Drama, Años 1910-1919, I Guer...",Drama bélico que ha pasado a la historia por s...,[1927: 2 Oscars: Mejor película y Efectos espe...,"7,3",1.361,https://pics.filmaffinity.com/wings-603432841-...,
2,116361,La coartada (Alibi),Alibi,1929,91 min.,Estados Unidos,Roland West,"[Chester Morris, Harry Stubbs, Mae Busch, Elea...","[Drama, Crimen, Mafia, Policíaco, Melodrama]",Chick Williams desea vengarse de quien lo envi...,[1928: 3 nominaciones al Oscar: Mejor película...,,,https://pics.filmaffinity.com/alibi-628810572-...,
3,660458,En el viejo Arizona,In Old Arizona,1928,95 min.,Estados Unidos,Irving Cummings,"[Warner Baxter, Edmund Lowe, Dorothy Burgess, ...",[Western],Cisco Kid (Warner Baxter) es un bandido del Vi...,"[1928: 1 Oscar: Mejor actor (Baxter), 5 nom., ...","5,6",52,https://pics.filmaffinity.com/in_old_arizona-9...,
4,309182,The Hollywood Revue of 1929,The Hollywood Revue of 1929,1929,118 min.,Estados Unidos,Charles Reisner,"[Conrad Nagel, Buster Keaton, Jack Benny, John...",[Musical],Musical por donde irán desfilando los grandes ...,[1929: Oscar: Nominada a la Mejor Película],"5,0",27,https://pics.filmaffinity.com/the_hollywood_re...,
558,845261,Historia de un matrimonio,Marriage Story,2019,136 min.,Estados Unidos,Noah Baumbach,"[Scarlett Johansson, Adam Driver, Laura Dern, ...","[Drama, Drama romántico, Teatro, Familia, Dram...","Charlie, un director de teatro neoyorquino y s...",[2019: Premios Oscar: Mejor actriz secundaria ...,"7,2",27.651,https://pics.filmaffinity.com/marriage_story-7...,
559,145446,1917,1917,2019,119 min.,Reino Unido,Sam Mendes,"[George MacKay, Dean-Charles Chapman, Mark Str...","[Bélico, Drama, I Guerra Mundial]","En lo más crudo de la Primera Guerra Mundial, ...","[2019: 3 Premios Oscar: Mejor fotografía, soni...","7,8",37.932,https://pics.filmaffinity.com/1917-960418215-l...,
560,169177,Érase una vez en... Hollywood,Once Upon a Time in... Hollywood,2019,165 min.,Estados Unidos,Quentin Tarantino,"[Leonardo DiCaprio, Brad Pitt, Margot Robbie, ...","[Thriller, Drama, Comedia, Años 60, Cine dentr...","Hollywood, años 60. La estrella de un western ...",[2019: Premios Oscar: Mejor actor secundario (...,"7,1",44.675,https://pics.filmaffinity.com/once_upon_a_time...,7
561,325196,El irlandés,The Irishman,2019,209 min.,Estados Unidos,Martin Scorsese,"[Robert De Niro, Al Pacino, Joe Pesci, Stephen...","[Thriller, Drama, Biográfico, Crimen, Mafia, V...",Frank Sheeran fue un veterano de la Segunda Gu...,"[2019: Premios Oscar: 0/10 nominaciones, inclu...","7,3",31.379,https://pics.filmaffinity.com/the_irishman-973...,8
562,274299,Le Mans '66,Ford v. Ferrari,2019,152 min.,Estados Unidos,James Mangold,"[Matt Damon, Christian Bale, Jon Bernthal, Cai...","[Drama, Acción, Biográfico, Coches / Automovil...",Se centra en un excéntrico y decidido equipo a...,[2019: 2 Premios Oscar: Mejor montaje y efecto...,"7,1",15.241,https://pics.filmaffinity.com/ford_v_ferrari-9...,


In [9]:
# Se utiliza el dataframe para exportar la información a CSV
df.to_csv('movie_data.csv', encoding='utf-8-sig')

In [10]:
# Se descargan todas las imágenes de las películas
for image_url in df["poster"]:
    load_requests(image_url)

C:/Users/a091b/images_FA/the_broadway_melody-974223234-large.jpg
C:/Users/a091b/images_FA/wings-603432841-large.jpg
C:/Users/a091b/images_FA/alibi-628810572-large.jpg
C:/Users/a091b/images_FA/in_old_arizona-995252878-large.jpg
C:/Users/a091b/images_FA/the_hollywood_revue_of_1929-338377848-large.jpg
C:/Users/a091b/images_FA/the_patriot-791348199-large.jpg
C:/Users/a091b/images_FA/seventh_heaven_7th_heaven-660930940-large.jpg
C:/Users/a091b/images_FA/the_racket-106449642-large.jpg
C:/Users/a091b/images_FA/you_can_t_take_it_with_you-642701189-large.jpg
C:/Users/a091b/images_FA/the_life_of_emile_zola-956734221-large.jpg
C:/Users/a091b/images_FA/the_great_ziegfeld-137216315-large.jpg
C:/Users/a091b/images_FA/mutiny_on_the_bounty-845467377-large.jpg
C:/Users/a091b/images_FA/it_happened_one_night-463534423-large.jpg
C:/Users/a091b/images_FA/cavalcade-638677271-large.jpg
C:/Users/a091b/images_FA/grand_hotel-709380464-large.jpg
C:/Users/a091b/images_FA/cimarron-701904076-large.jpg
C:/Users/a091

C:/Users/a091b/images_FA/the_magnificent_ambersons-666570410-large.jpg
C:/Users/a091b/images_FA/the_pied_piper-313899578-large.jpg
C:/Users/a091b/images_FA/the_pride_of_the_yankees-299498318-large.jpg
C:/Users/a091b/images_FA/random_harvest-386951375-large.jpg
C:/Users/a091b/images_FA/the_talk_of_the_town-560864389-large.jpg
C:/Users/a091b/images_FA/wake_island-312356329-large.jpg
C:/Users/a091b/images_FA/yankee_doodle_dandy-437959408-large.jpg
C:/Users/a091b/images_FA/blossoms_in_the_dust-807203823-large.jpg
C:/Users/a091b/images_FA/citizen_kane-217208961-large.jpg
C:/Users/a091b/images_FA/here_comes_mr_jordan-582736687-large.jpg
C:/Users/a091b/images_FA/hold_back_the_dawn-892439724-large.jpg
C:/Users/a091b/images_FA/the_little_foxes-752428013-large.jpg
C:/Users/a091b/images_FA/the_maltese_falcon-111426057-large.jpg
C:/Users/a091b/images_FA/one_foot_in_heaven-521650481-large.jpg
C:/Users/a091b/images_FA/sergeant_york-198693587-large.jpg
C:/Users/a091b/images_FA/suspicion-212883262-lar

C:/Users/a091b/images_FA/anatomy_of_a_murder-592643953-large.jpg
C:/Users/a091b/images_FA/the_diary_of_anne_frank-697697755-large.jpg
C:/Users/a091b/images_FA/the_nun_s_story-335671837-large.jpg
C:/Users/a091b/images_FA/room_at_the_top-371636180-large.jpg
C:/Users/a091b/images_FA/the_deer_hunter-262693807-large.jpg
C:/Users/a091b/images_FA/annie_hall-798503064-large.jpg
C:/Users/a091b/images_FA/rocky-297722683-large.jpg
C:/Users/a091b/images_FA/one_flew_over_the_cuckoo_s_nest-886138071-large.jpg
C:/Users/a091b/images_FA/the_godfather_part_ii-617901053-large.jpg
C:/Users/a091b/images_FA/the_sting-433653100-large.jpg
C:/Users/a091b/images_FA/the_godfather-488102675-large.jpg
C:/Users/a091b/images_FA/the_french_connection-860263784-large.jpg
C:/Users/a091b/images_FA/patton-953905809-large.jpg
C:/Users/a091b/images_FA/midnight_cowboy-956238878-large.jpg
C:/Users/a091b/images_FA/coming_home-780424242-large.jpg
C:/Users/a091b/images_FA/heaven_can_wait-888279322-large.jpg
C:/Users/a091b/image

C:/Users/a091b/images_FA/the_crying_game-313067334-large.jpg
C:/Users/a091b/images_FA/a_few_good_men-646229005-large.jpg
C:/Users/a091b/images_FA/howards_end-457044974-large.jpg
C:/Users/a091b/images_FA/scent_of_a_woman-162635554-large.jpg
C:/Users/a091b/images_FA/beauty_and_the_beast-152980110-large.jpg
C:/Users/a091b/images_FA/bugsy-629749660-large.jpg
C:/Users/a091b/images_FA/j_f_k_jfk-515455744-large.jpg
C:/Users/a091b/images_FA/the_prince_of_tides-579669069-large.jpg
C:/Users/a091b/images_FA/the_godfather_part_iii-171971720-large.jpg
C:/Users/a091b/images_FA/ghost-419228198-large.jpg
C:/Users/a091b/images_FA/goodfellas-722948658-large.jpg
C:/Users/a091b/images_FA/awakenings-753003216-large.jpg
C:/Users/a091b/images_FA/born_on_the_fourth_of_july-674714087-large.jpg
C:/Users/a091b/images_FA/dead_poets_society-945987795-large.jpg
C:/Users/a091b/images_FA/field_of_dreams-874774489-large.jpg
C:/Users/a091b/images_FA/my_left_foot-568597624-large.jpg
C:/Users/a091b/images_FA/slumdog_mill

C:/Users/a091b/images_FA/extremely_loud_and_incredibly_close-569605369-large.jpg
C:/Users/a091b/images_FA/the_social_network-421032493-large.jpg
C:/Users/a091b/images_FA/the_fighter-295947851-large.jpg
C:/Users/a091b/images_FA/black_swan-861293346-large.jpg
C:/Users/a091b/images_FA/inception-652954101-large.jpg
C:/Users/a091b/images_FA/toy_story_3-691147043-large.jpg
C:/Users/a091b/images_FA/127_hours-846868462-large.jpg
C:/Users/a091b/images_FA/winter_s_bone-270791611-large.jpg
C:/Users/a091b/images_FA/true_grit-556962376-large.jpg
C:/Users/a091b/images_FA/the_kids_are_all_right-286479351-large.jpg
C:/Users/a091b/images_FA/avatar-208925608-large.jpg
C:/Users/a091b/images_FA/district_9-116196176-large.jpg
C:/Users/a091b/images_FA/an_education-624223159-large.jpg
C:/Users/a091b/images_FA/the_blind_side-537970540-large.jpg
C:/Users/a091b/images_FA/a_serious_man-606969159-large.jpg
C:/Users/a091b/images_FA/inglourious_basterds-976286054-large.jpg
C:/Users/a091b/images_FA/precious_based_on